In [61]:
import numpy as np
from abc import ABC, abstractmethod 

In [69]:
def sumit(A, B):
    return np.sum(A * B)

def conv(A, K):
    dima = A.shape[0]
    dimk = K.shape[0]
    dimz = dima - dimk + 1
    Z = np.zeros((dimz, dimz))
    for i in range(dimz):
        for j in range(dimz):
            Z[i,j] = sumit(A[i:i+dimk, j:j+dimk], K) 
    return Z

In [91]:
def relU(x):
    return x if x >= 0 else 0

def relU_prime(x):
    return 1 if x >= 0 else 0

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [204]:
class Layer(ABC): 

    @property
    def size(self):
        pass

    @abstractmethod
    def feed_forward(self, alpha, A, dZ):
        pass

    @abstractmethod
    def backprop(self, dZ):
        pass


class ConvLayer(Layer):
    def __init__(self, input_size, filter_size, g=relU, dg=relU_prime):
        self.K = np.random.rand(filter_size, filter_size)
        self.b = 1
        self.g = np.vectorize(g)
        self.dg = np.vectorize(dg)
        self._output_size = input_size - filter_size + 1
        self._size = (input_size - filter_size + 1) ** 2; 

    @property
    def size(self):
        return self._size
    
    @size.setter
    def size(self, value):
        self._size = value

    def feed_forward(self, A):
        self.Z = conv(A, self.K) + self.b
        self.A = self.g(self.Z)
        return self.A
    
    def grad_k(self, A, DZ):
        return conv(A, DZ)
    
    def backprop(self, dA):
        dimk = self.K.shape[0]
        dZ = dA.reshape(self._output_size, self._output_size) * self.dg(self.Z)
        self.dA = conv(np.pad(dZ, dimk-1), np.flip(self.K))
        return self.dg(self.dA)
    
    def incoming_layer(self):
        return self._incoming_layer



class DenseLayer(Layer):
    def __init__(self, input_size, size, g = sigmoid, dg = sigmoid_prime):
        self.B = np.random.rand(size)
        self.W = np.random.randn(size, input_size)
        self._size = size
        self.dg = dg
        self.g = g

    @property
    def size(self):
        return self._size
    
    @size.setter
    def size(self, value):
        self._size = value

    def feed_forward(self, A):
        self.Z = self.W @ A.flatten() + self.B
        self.A = self.g(self.Z)
        return self.A

    def dLdA(self, dZ):
        return self.K @ dZ

    def backprop(self, dA):
        dZ = dA * self.dg(self.Z)
        return (self.W.T @ dZ)
    
    def incoming_layer(self):
        return self._incoming_layer


class InputLayer(Layer):
    def __init__(self, size):
        self._size = size

    @property
    def size(self):
        return self._size
    
    @size.setter
    def size(self, value):
        self._size = value

    def feed_forward(self, A):
        self.A = A
        return self.A
    
    def backprop(self, dA):
        pass

In [208]:
layer1 = InputLayer(size = 5 * 5)
layer2 = ConvLayer(input_size=5, filter_size=2)
layer3 = DenseLayer(input_size=layer2.size, size=4)
layer4 = DenseLayer(input_size=layer3.size, size=5)

model = [layer1, layer2, layer3, layer4]

a = np.arange(5 * 5).reshape(5,5)
for layer in model:
    a = layer.feed_forward(a)
    print(a)

dA = np.random.randn(4)
for layer in reversed(model):
    dA = layer.backprop(dA)


[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
[[ 9.39514525 12.03135532 14.6675654  17.30377547]
 [22.57619562 25.21240569 27.84861576 30.48482584]
 [35.75724598 38.39345606 41.02966613 43.6658762 ]
 [48.93829635 51.57450642 54.2107165  56.84692657]]
[1.00000000e+00 9.99998988e-01 1.00000000e+00 2.31698804e-10]
